In [1]:
%%capture
import sys
!pip install tensorflow-addons
!git clone https://github.com/ICascha/QuantGANs-replication.git
sys.path.append('/content/QuantGANs-replication/')

In [2]:
from backend.preprocessing import *
from backend.metrics import *
from backend.gan import CGAN
from backend.tcn import make_TCN

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.random import normal
from tensorflow.keras.utils import Progbar
from datetime import datetime, timedelta

In [3]:
batch_size = 64
n_batches = 5000
additional_d_steps = 0

In [4]:
# crsp dataset of jpmc returns and sp500 returns, obtained from wrds
df = pd.read_csv('/content/jpmc_crsp.csv', parse_dates=['date'], usecols=['RET', 'sprtrn', 'date'])
df = df[(df['date'] >= datetime.strptime('06 03 69', '%d %m %y')) & (df['date'] <= datetime.strptime('01 1 05', '%d %m %y'))]

log_returns = df[['RET', 'sprtrn']].astype('float64').to_numpy()

In [5]:
s1 = StandardScaler()

# receptive field size
rfs = 253

# pre-processing pipeline
log_returns_preprocessed = s1.fit_transform(log_returns)
log_returns_rolled = rolling_window(log_returns_preprocessed, rfs)

In [6]:
dilations = 2**(np.arange(6))
n_filters = 60

discriminator = make_TCN(dilations, n_filters, 0, False, True, False, [2, rfs, 1], block_size=4)
generator = make_TCN(dilations, n_filters, 0, True, False, False, [2*2, None, 3], halve_output_series=True)

In [7]:
gan = CGAN(discriminator, generator, rfs, lr_d=3e-4, lr_g=1e-4)

In [8]:
data = np.expand_dims(np.moveaxis(np.moveaxis(log_returns_rolled, 0, 1), 1, 2), -1).astype('float32')
batch_size = 128
n_batches = 2000
additional_d_steps = 0

gan.train(data, batch_size, n_batches, additional_d_steps)

2000/2000 [==============================] - 2363s 1s/step


In [9]:
%%capture
generator.save('trained_jpmc_generator_05')
!zip -r trained_jpmc_generator_05.zip trained_jpmc_generator_05/

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/THESIS/JPMC_generator_20/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/THESIS/JPMC_generator_20/assets


In [10]:
files.download('trained_jpmc_generator_05.zip')